In [ ]:
import requests
import json
import urllib
import  urllib.parse
import pandas as pd
import getpass
from requests.auth import HTTPBasicAuth
token = ''

In [ ]:
from ikewaipy.ikewai import Ikewai

In [ ]:
#Setup a client object for connecting to Ike Wai so we can get auth token
ike = Ikewai(username='seanbc')

In [ ]:
#login with your UH credentials
ike.login()

In [ ]:
#this is how we access our Auth Token
ike.token

In [ ]:
#This is how we use the ikewaipy search for JSON metadata objects - similiar to listMetadata
ike.searchMetadata("{'name':'Variable','value.variable_name':'Temperature'}", 1)

In [ ]:
#HTTP REST call to meta API with a mongo style query
def listMetadata(token, query="", limit=10, offset=0):
    safe_query = urllib.parse.quote(query.encode('utf8'))
    headers = {
        'authorization': "Bearer " + token,
        'content-type': "application/json",
    }
    res = requests.get('https://'+endpoint+'/meta/v2/data?q='+safe_query+'&limit='+str(limit)+'&offset='+str(offset), headers=headers,verify=False)
    resp = json.loads(res.content)
    return resp['result']

In [ ]:
#HTTP API call to meta to fetch a single metadata JSON document by it's uuid
def getMetadata(token, uuid):
    headers = {
        'authorization': "Bearer " + token,
        'content-type': "application/json",
    }
    res = requests.get('https://'+endpoint+'/meta/v2/data/'+uuid, headers=headers,verify=False)
    resp = json.loads(res.content)
    return resp['result']

In [ ]:
#HTTPS API call to update an existing metadata JSON document - note you have to pass every field as this is not a PATCH
def updateMetadata(token, uuid, data):
    headers = {
        'authorization': "Bearer " + token,
        'content-type': "application/json",
    }
    res = requests.post('https://'+endpoint+'/meta/v2/data/'+uuid, json=data, headers=headers,verify=False)
    resp = json.loads(res.content)
    return resp

In [ ]:
#CREATE a new metadata document
def createMetadata(token, data):
    headers = {
        'authorization': "Bearer " + token,
        'content-type': "application/json",
    }
    res = requests.post('https://'+endpoint+'/meta/v2/data/', json=json.loads(data), headers=headers,verify=False)
    print(res.content)
    resp = json.loads(res.content)
    return resp

In [ ]:
#permission needs to be READ READ-WRITE or ALL
def updateMetadataPem(token, uuid, username, permission):
    headers = {
        'authorization': "Bearer " + token,
        'content-type': "application/json",
    }
    data = {'permission':permission}
    res = requests.post('https://'+endpoint+'/meta/v2/data/'+uuid+'/pems/'+username, json=data, headers=headers,verify=False)
    resp = json.loads(res.content)
    return resp
    

In [ ]:
def getMetadataPems(token,uuid):
    headers = {
        'authorization': "Bearer " + token,
        'content-type': "application/json",
    }
    res = requests.get('https://'+endpoint+'/meta/v2/data/'+uuid+'/pems/', headers=headers,verify=False)
    resp = json.loads(res.content)
    return resp

In [ ]:
#load csv in a dataframe example and see dataframe
df1 = pd.read_csv('GPS.csv')
df1

In [ ]:
#Parse a CSV file and convert it to JSON metadata documents

#read in csv file to a pandas dataframe
df1 = pd.read_csv('GPS.csv')

#set static json body values and permsissions
#users needs to exist in agave
body={}
pem1={}
pem1['username']= 'seanbc'
pem1['permission']='ALL'
pem2={}
pem2['username']= 'jgeis'
pem2['permission']='ALL'
pem4={}
pem4['username']= 'ikewai-admin'
pem4['permission']='ALL'
pem5={}
pem5['username']= 'public'
pem5['permission']='READ'

#the name field should be the larger document type we want - so this should be akin to the CSV file - we will use this for searching for these document types
body['name'] = "Micro-GPS"

#add our permissions to the document body so they are set at creation
body['permissions']=[pem1,pem2,pem4,pem5]


#should loop through each dataframe row convert to json 
i=0
ls = [0,1,2]
for i in df1.index:
    j = df1.loc[i].to_json()
    js = json.loads(j)
    #This stores a GeoJSON object in the value.loc field - in Ike Wai this has a spatial index on it in mongodb
    #ONLY DO THIS FOR georeferenced documents - so probably just one CSV
    js['loc'] = {"type":"Point", "coordinates":[js['Longitude'],js['Lattitude']]}
    body['value'] = js
    body['geospatial']= True;
    #preview our JSON document
    print(body)
    #create the document in Ike Wai - 
    #createMetadata(ike.token,json.dumps(body))
    